In [ ]:
import pandas as pd

import configparser
import os
import glob
from bs4 import BeautifulSoup
from sqlalchemy import create_engine

import hashlib

config = configparser.ConfigParser()
config.read(os.path.join(os.path.expanduser('~'), '.config', 'me.ini'))

engine = create_engine(config['db']['url'])
pd.options.display.max_rows = 1000

# Accounts

In [ ]:
accounts = pd.read_csv('accounts.csv')
accounts.to_sql('accounts', engine, if_exists='replace')

# CGD

In [ ]:
with open('valuescgd.txt', encoding="iso-8859-1") as cgd:
    values = cgd.readlines()
values = ''.join(values).split('------')

In [ ]:
positions = [
    'Descrição',
    
    'Montante do juro',
    'Montante do imposto',
    'Montante da transferência',
    'Montante original',
    'Montante',
    
    'Data valor',
    'Data do movimento',
    'Tipo de movimento',
    'Saldo contabilístico após movimento',
    'Saldo disponível após movimento',
    'Comentário cliente',
    'Entidade',
    'Número do cartão',
    'N.º Identificação SIBS',
    'Meio de pagamento utilizado',
    'Data início de cálculo do juro',
    'Data fim de cálculo do juro',
    'Conta juros credores',
    'Local',
    'Operação efectuada em',
    'Referência',
    'Valor cativo após movimento',
    'Conta destino',
    'Nº de Transferência',
    'Taxa de câmbio',
    'Contravalor',
    'Descritivo conta destino',
    'Conta destino',
    'Tipo serviço',
    'Finalidade',
    'Nome Ordenante',
    'Banco Ordenante',
    'Swift Ordenante',
    'Número do cheque',
    'Nome do destinatário',
    'Swift do destinatário',
    'Iban do destinatário',
    'Motivo'
]

In [ ]:
# check if there are any positions not in the list
for val in values:
    for line in val.splitlines():
        found_pos = False
        for pos in positions:
            if pos in line:
                found_pos = True
                
        if not found_pos and line:
            print(line)

In [ ]:
# Create a list of values
mov_id = 1
newvalues = []
for val in values:
    mov_type = None
    for line in val.splitlines():
        for pos in positions:
            # print(str(mov_id) + ' ' + pos + ' - ' + line)
            if pos in line:
                newvalues.append({
                    'type': pos.strip(),
                    'value': line.replace(pos, '').strip(),
                    'movement_id': mov_id
                })
                break
    mov_id += 1

valdf = pd.DataFrame(newvalues)
print('List has {} and data frame has {} ({} unique).'.format(len(values), len(valdf), len(valdf.movement_id.unique())))

In [ ]:
# check how many values each position has
global_pos = []
unique_movs = len(valdf.movement_id.unique())
for pos in positions:
    poslen = len(valdf[valdf['type'] == pos])
    #print('position "{}" has {} values.'.format(pos, poslen))
    
    if poslen == unique_movs:
        global_pos.append(pos)

In [ ]:
# create the main table
cgdmovements = valdf[valdf['type'].isin(global_pos)].pivot(index='movement_id', columns='type', values='value')

# better columns names
cgdmovements.columns = [
    'comment',
    'movement_date',
    'value_date',
    'description',
    'ammount',
    'balance',
    'available_balance',
    'movement_type']

# convert a few values
cgdmovements.value_date = pd.to_datetime(cgdmovements.value_date, format="%d-%m-%Y")

# add a midnight hour to movements that do not have it
cgdmovements.ix[cgdmovements.movement_date.str.contains('[0-9]{2}-[0-9]{2}-[0-9]{4}\Z'), 'movement_date'] = \
    cgdmovements.ix[cgdmovements.movement_date.str.contains('[0-9]{2}-[0-9]{2}-[0-9]{4}\Z'), 'movement_date'] + ' 00:00'
cgdmovements.movement_date = pd.to_datetime(cgdmovements.movement_date, format="%d-%m-%Y %H:%M")

# convert currency
cgdmovements.ix[cgdmovements.movement_type == 'Débito', 'ammount'] = \
    '-' + cgdmovements.ix[cgdmovements.movement_type == 'Débito', 'ammount']

cgdmovements.ammount = cgdmovements.ammount.str.replace(' EUR', '')
cgdmovements.ammount = cgdmovements.ammount.str.replace('.', '')
cgdmovements.ammount = cgdmovements.ammount.str.replace(',', '.')
cgdmovements.ammount = pd.to_numeric(cgdmovements.ammount)


cgdmovements.balance = cgdmovements.balance.str.replace(' EUR', '')
cgdmovements.balance = cgdmovements.balance.str.replace('.', '')
cgdmovements.balance = cgdmovements.balance.str.replace(',', '.')
cgdmovements.balance = pd.to_numeric(cgdmovements.balance)


cgdmovements.available_balance = cgdmovements.available_balance.str.replace(' EUR', '')
cgdmovements.available_balance = cgdmovements.available_balance.str.replace('.', '')
cgdmovements.available_balance = cgdmovements.available_balance.str.replace(',', '.')
cgdmovements.available_balance = pd.to_numeric(cgdmovements.available_balance)

# remove columns not needed, and sort it.
cgdmovements = cgdmovements[cgdmovements.columns[1:]]
cgdmovements.sort_values(by='value_date', inplace=True)
cgdmovements['account_id'] = 1
cgdmovements['movement_id'] = cgdmovements.index


#cgdmovements.to_sql('account_movements', engine, if_exists='append')
#valdf.to_sql('account_details', engine, if_exists='append', index=False)

# Totta

In [ ]:
max_mov_id = cgdmovements.index.max() + 1

tottamov = pd.read_csv('tottavalues.csv', sep='\t')

tottamov.columns = [
    'movement_date',
    'value_date',
    'description',
    'ammount',
    'available_balance']

tottamov.movement_date = pd.to_datetime(tottamov.movement_date, format="%d-%m-%Y")
tottamov.value_date = pd.to_datetime(tottamov.value_date, format="%d-%m-%Y")

tottamov.ammount = tottamov.ammount.str.replace(',', '.')
tottamov.ammount = tottamov.ammount.str.replace(' ', '')
tottamov.ammount = pd.to_numeric(tottamov.ammount)

tottamov.available_balance = tottamov.available_balance.str.replace(',', '.')
tottamov.available_balance = tottamov.available_balance.str.replace(' ', '')
tottamov.available_balance = pd.to_numeric(tottamov.available_balance)

tottamov.ix[tottamov.ammount < 0, 'movement_type'] = 'Débito'
tottamov.ix[tottamov.ammount > 0, 'movement_type'] = 'Crédito'

tottamov['movement_id'] = range(max_mov_id, max_mov_id + len(tottamov))
tottamov['balance'] = tottamov.available_balance
tottamov['account_id'] = 10

#tottamov.to_sql('account_movements', engine, if_exists='append', index=False)

# Interest from certificados

In [ ]:
max_mov_id = tottamov.index.max() + 1

aforro = pd.read_sql('aforro', engine)

values = []
for account in aforro.subscription_number.unique():
    new_df = aforro[aforro.subscription_number == account].copy()
    new_df['interest'] = new_df.value - new_df.shift().value
    values.append(new_df)
    
interest = pd.concat(values)
interest = interest[(interest['interest'] > 0) & (interest['product'] != 'CT CTPM')]
interest['abbreviation'] = interest.subscription_number.str.replace('-1', '')

interest = pd.merge(interest, accounts[['account_id', 'abbreviation']], how='left', on='abbreviation')
interest['movement_date'] = interest['value_date']
interest['description'] = 'Juros ' + interest['subscription_number']
interest['ammount'] = interest['interest']
interest['balance'] = interest['value']
interest['available_balance'] = interest['value']
interest['movement_type'] = 'Crédito'

interest = interest[['movement_date', 'value_date', 'description', 'ammount', 'balance', 'available_balance', 'movement_type', 'account_id']]

interest['movement_id'] = range(max_mov_id, max_mov_id + len(interest))

#interest.to_sql('account_movements', engine, if_exists='append', index=False)

# Other movements

In [ ]:
max_mov_id = interest.index.max() + 1

other = pd.read_csv('othermovements.csv')

other['movement_id'] = range(max_mov_id, max_mov_id + len(other))
#df.to_sql('account_movements', engine, if_exists='append', index=False)

# Meals

In [ ]:
max_mov_id = other.index.max() + 1

meals = pd.read_csv('meals.csv')
meals['movement_id'] = range(max_mov_id, max_mov_id + len(meals))
#df.to_sql('account_movements', engine, if_exists='append', index=False)

# CGD CC

In [ ]:
max_mov_id = meals.index.max() + 1

tables = []
for fl in glob.glob('cgd/cartao*'):
    print('processing file ' + str(fl))
    with open(fl) as cartfile:
        b = BeautifulSoup(''.join(cartfile.readlines()), "lxml")

        # credits
        tbl = b.find("table", { "id" : "detailMOV_CREDDEBDIF_1" })
        if tbl:
            df = pd.read_html(tbl.prettify())[0]
            df.columns = ['movement_date', 'value_date', 'description', 'a', 'ammount', 'b', 'c']
            df.ammount = df.ammount / 100
            df['movement_type'] = 'Crédito'
            df['account_id'] = 7

            tables.append(df[['movement_date', 'value_date', 'description', 'ammount', 'movement_type', 'account_id']])
            

        # debits
        tbl = b.find("table", { "id" : "detailMOV_CREDDEBDIF_0" })
        if tbl:
            df = pd.read_html(tbl.prettify())[0]
            df.columns = ['movement_date', 'value_date', 'description', 'ammount', 'a', 'b', 'c']
            df.ammount = 0 - (df.ammount / 100)
            df['movement_type'] = 'Débito'
            df['account_id'] = 7

            tables.append(df[['movement_date', 'value_date', 'description', 'ammount', 'movement_type', 'account_id']])


ccdf = pd.concat(tables)
ccdf.movement_date = pd.to_datetime(ccdf.movement_date, format="%d-%m-%Y")
ccdf.value_date = pd.to_datetime(ccdf.value_date, format="%d-%m-%Y")
ccdf.sort_values(by='movement_date', inplace=True)
ccdf.reset_index(inplace=True)
ccdf = ccdf[['movement_date', 'value_date', 'description', 'ammount', 'movement_type', 'account_id']]


bal_series = []
balance = 0
for index, row in ccdf.iterrows():
    balance += round(row.ammount, 2)
    bal_series.append(balance)

ccdf['balance'] = pd.Series(bal_series)
ccdf['available_balance'] = ccdf.balance

ccdf['movement_id'] = range(max_mov_id, max_mov_id + len(ccdf))
#df.to_sql('account_movements', engine, if_exists='append', index=False)

In [ ]:
df = pd.concat([cgdmovements, tottamov, interest, other, meals, ccdf])

df['hash'] = df.apply(lambda x:hashlib.md5('{}|{}|{}|{}|{}|{}|{}|{}'.format(
        x['movement_date'], 
        x['value_date'], 
        x['description'], 
        x['ammount'], 
        x['balance'], 
        x['available_balance'], 
        x['movement_type'], 
        x['account_id']).encode('utf-8')).hexdigest(), axis=1)

df.to_sql('account_movements', engine, if_exists='replace', index=False)
valdf.to_sql('account_details', engine, if_exists='replace', index=False)